In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json

with open(r'/content/intents.json') as f:
    data = json.load(f)

df_chat = pd.DataFrame(data['intents'])

In [ ]:

df_chat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tag        80 non-null     object
 1   patterns   80 non-null     object
 2   responses  80 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


In [ ]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df_chat)):
    ptrns = df_chat[df_chat.index == i]['patterns'].values[0]
    rspns = df_chat[df_chat.index == i]['responses'].values[0]
    tag = df_chat[df_chat.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

In [ ]:
df_chat = pd.DataFrame.from_dict(dic)

In [ ]:
df_chat

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
227,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or beha..."
228,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
229,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
230,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


###**Tokenizer**###

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df_chat['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df_chat['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df_chat['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

number of unique words =  303
X shape =  (232, 18)
y shape =  (232,)
num of classes =  80


###**Pre-trained tokenizer Weights**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer

# Instantiate the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the patterns column and create X
ptrn2seq = [tokenizer.encode(pattern, max_length=32, padding='max_length', truncation=True) for pattern in df_chat['patterns']]
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

# Create label encoder and encode y
lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df_chat['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))


X shape =  (232, 32)
y shape =  (232,)
num of classes =  80


###**LSTM Model**###

In [ ]:
# Build the model architecture
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=100, mask_zero=True),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         5026500   
                                                                 
 lstm (LSTM)                 (None, None, 32)          17024     
                                                                 
 layer_normalization (LayerN  (None, None, 32)         64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, None, 32)          8320      
                                                                 
 layer_normalization_1 (Laye  (None, None, 32)         64        
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

###**BI-LSTM Model**###

In [ ]:
# Build the model architecture
bi_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

# Compile the model
bi_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         30400     
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 64)         34048     
 nal)                                                            
                                                                 
 layer_normalization_4 (Laye  (None, None, 64)         128       
 rNormalization)                                                 
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 layer_normalization_5 (Laye  (None, 64)               128       
 rNormalization)                                      

###**BERT Mdoel**###

In [ ]:
from transformers import TFRobertaModel, RobertaTokenizer
# Instantiate the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the patterns column and create X
# X = [tokenizer.encode(pattern, max_length=32, padding='max_length', truncation=True) for pattern in df_chat['patterns']]
ptrn2seq = [tokenizer.encode(pattern, max_length=32, padding='max_length', truncation=True) for pattern in df_chat['patterns']]
X = pad_sequences(ptrn2seq, padding='post')
X = np.array(X)
# Create label encoder and encode y
lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df_chat['tag'])
y=np.array(y)
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

# Load the pre-trained BERT model and set it to trainable
bert_model = TFRobertaModel.from_pretrained('roberta-base')
bert_model.trainable = True

# Build the model architecture
input_ids = tf.keras.layers.Input(shape=(32,), dtype=tf.int32, name='input_ids')
outputs = bert_model(input_ids)[0]
outputs = tf.keras.layers.Dense(128, activation='relu')(outputs)
outputs = tf.keras.layers.Dropout(0.2)(outputs)
outputs = tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')(outputs)

bert_model = tf.keras.Model(inputs=input_ids, outputs=outputs)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# history = bert_model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)



y shape =  (232,)
num of classes =  80


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


###**Model Training**###

In [ ]:
model_history = bi_model.fit(X, y,
                          batch_size=5,
                          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)],
                          epochs=100)

Epoch 1/100
47/47 [==============================] - 27s 69ms/step - loss: 4.8792 - accuracy: 0.0043
Epoch 2/100
47/47 [==============================] - 6s 126ms/step - loss: 3.1405 - accuracy: 0.2198
Epoch 3/100
47/47 [==============================] - 2s 45ms/step - loss: 2.1358 - accuracy: 0.4698
Epoch 4/100
47/47 [==============================] - 2s 43ms/step - loss: 1.5288 - accuracy: 0.6853
Epoch 5/100
47/47 [==============================] - 2s 44ms/step - loss: 0.9695 - accuracy: 0.8578
Epoch 6/100
47/47 [==============================] - 2s 49ms/step - loss: 0.6853 - accuracy: 0.9267
Epoch 7/100
47/47 [==============================] - 4s 77ms/step - loss: 0.4169 - accuracy: 0.9741
Epoch 8/100
47/47 [==============================] - 2s 43ms/step - loss: 0.3107 - accuracy: 0.9698
Epoch 9/100
47/47 [==============================] - 2s 43ms/step - loss: 0.2439 - accuracy: 0.9871
Epoch 10/100
47/47 [==============================] - 2s 44ms/step - loss: 0.2027 - accuracy: 0.98

###**Model Saving**###

In [ ]:
bi_model.save('bi_model.h5')

###**Gradio Interface**###

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6

In [ ]:
model = tf.keras.models.load_model('/content/bi_model.h5')

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import random

def model_response(query):
    # Tokenize and encode the query as input IDs using the tokenizer
    input_ids = tokenizer.encode(query, add_special_tokens=True)
    # Convert the input IDs to a numpy array and reshape it for the model
    input_ids = np.array(input_ids).reshape(1, -1)
    # Pass the input IDs to the model and get the predicted output
    outputs = model.predict(input_ids)
    # Extract the predicted class index from the outputs
    predicted = np.argmax(outputs)
    # Get the corresponding response from the dataframe
    tag = lbl_enc.inverse_transform([predicted])[0]
    responses = df_chat[df_chat['tag'] == tag]['responses'].values[0]
    response = random.choice(responses)
    # return responses
    return response

iface = gr.Interface(fn=model_response, inputs="text", outputs="text",
                     title="Chatbot", description="Enter a message to get a response from the chatbot.")
iface.launch()



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>